In [1]:
import tensorflow as tf
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we can use again a convenient built-in function to import a graph_def into the 
    # current default Graph
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(
            graph_def, 
            return_elements=None, 
            name="prefix", 
            op_dict=None, 
            producer_op_list=None
        )
    return graph

# We use our "load_graph" function
graph = load_graph("./Model/output_graph.pb")

# We can verify that we can access the list of operations in the graph
for op in graph.get_operations():
    print(op.name)     # <--- printing the operations snapshot below
    # prefix/Placeholder/inputs_placeholder
    # ...
    # prefix/Accuracy/predictions

AttributeError: module 'tensorflow' has no attribute 'gfile'

In [12]:
y_pred = graph.get_tensor_by_name("prefix/time_distributed_out/Reshape_1:0")

In [48]:
x = graph.get_tensor_by_name("prefix/codes_input:0")
z = graph.get_tensor_by_name("prefix/numeric_input:0")

In [158]:
import numpy as np
import pandas as pd
import random
from random import randint
from random import uniform
from sklearn.model_selection import train_test_split
import pickle

def get_rand_visit_codes():
    #num_of_codes_in_visit = randint(1,5)
    #visit_c = random.sample(range(0,68), num_of_codes_in_visit)
    num_of_codes_in_visit = 2
    visit_c = []
    #print (visit_c)
    return visit_c
ite_rand = 0
def get_rand_visit_numerics():
    #[Diastolic,Systolic, pulse]
    global ite_rand
    if ite_rand == 0:
        visit_n = [uniform(60,90),uniform(90,160),uniform(60,100)]
    else:
        visit_n = [uniform(90,110),uniform(160,180),uniform(100,120)]
    #print (visit_n)
    return visit_n

def create_patient():
    num_of_visits = randint(1,25)
    global ite_rand
    if ite_rand == 0:
        ite_rand = 1
    else:
        ite_rand = 0
    anomaly_patient_count = 0
    patient_c = []
    for k in range(0,num_of_visits,1):
        patient_c.append(get_rand_visit_codes())
    #print (patient_c)

    patient_n = []
    for k in range(0,num_of_visits,1):
        visit_numerics = get_rand_visit_numerics()
        patient_n.append(visit_numerics)
        anomaly_patient_count = anomaly_patient_count + validate_numerics_for_anomaly(visit_numerics)
        
    #print ("New patient "+str(len(target))+", # of visits: "+str(num_of_visits))
    #print ("Number of anomalies: "+ str(anomaly_patient_count))
    target = 0
    if anomaly_patient_count/num_of_visits > 0.7:
        target = 1
        #print ("Target: "+ str(1))
    else:
        target = 0
        #print ("Target: "+ str(0))
    #print("Target value in list: "+str(target[len(target)-1]))
    return [patient_c,patient_n,None,anomaly_patient_count, target]

def validate_numerics_for_anomaly(numerics):
    #[Diastolic,Systolic, pulse]
    if (numerics[0] >= 100 or numerics[0] <= 55):
        return 1
    elif (numerics[1] >= 170 or numerics[1] <= 85):
        return 1
    elif (numerics[2] >= 110 or numerics[2] <= 55):
        return 1
    else:
        return 0

In [165]:
patient_1 = create_patient()
print("patient_codes: "+ str(patient_1[0]))
print("patient_nums: "+ str(patient_1[1]))
print("patient_anomalies: " +str(patient_1[3]))
print("patient_target: " +str(patient_1[4]))

patient_codes: [[], [], [], [], [], [], [], [], []]
patient_nums: [[102.46265960976592, 161.84189726360853, 115.39472246221708], [106.2801928167619, 167.74015650997114, 111.20465593331818], [95.29400881447118, 170.94809228821745, 100.72956559377283], [99.16243317265884, 168.09953449994546, 107.04374832240916], [102.76409386803302, 169.57890467816887, 101.62492754919089], [98.91324652640172, 171.56279669060177, 103.69519025217396], [98.16321258143104, 162.80701742550076, 105.93718436218849], [92.28354669852581, 179.31458663745056, 103.57760785526487], [107.43176399679531, 164.16805439393883, 118.48160719314988]]
patient_anomalies: 7
patient_target: 1


In [166]:
feed_dict_testing = {
    x: [patient_1[0]],
    z: [patient_1[1]]
}
sess= tf.Session(graph=graph)
result=sess.run(y_pred, feed_dict=feed_dict_testing)

In [167]:
print(result)

[[[0.9258097]]]


In [168]:
patient_2 = create_patient()
print("patient_codes: "+ str(patient_2[0]))
print("patient_nums: "+ str(patient_2[1]))
print("patient_anomalies: " +str(patient_2[3]))
print("patient_target: " +str(patient_2[4]))

patient_codes: [[], [], [], [], [], [], [], [], [], [], [], [], []]
patient_nums: [[81.58675497124098, 109.32976295134867, 84.14181249303995], [69.70657866154629, 159.0450610530469, 73.57852377636516], [75.7933196280955, 139.90656238995777, 63.1498977183786], [62.28691804163644, 98.95535128201709, 72.2688770696065], [82.74432291361533, 131.25142495212128, 74.32070503515453], [85.97603772498697, 107.22965844045324, 93.10421161588653], [65.8347055907382, 128.33669440414138, 88.05376522913136], [77.39557599293364, 145.62697610410976, 75.16783649729588], [70.86042000374712, 140.62867483053157, 61.439735795079514], [66.96611704174586, 153.86865525875675, 90.39236993622174], [73.40479378358742, 103.96368483319577, 60.71693068884196], [80.57610118482674, 120.7704191825938, 79.55084367645671], [67.82834755944212, 127.40556418491008, 93.80747288074107]]
patient_anomalies: 0
patient_target: 0


In [169]:
feed_dict_testing = {
    x: [patient_2[0]],
    z: [patient_2[1]]
}
sess= tf.Session(graph=graph)
result=sess.run(y_pred, feed_dict=feed_dict_testing)

In [170]:
print(result)

[[[0.24745676]]]
